In [18]:
'Harry Potter and the Order of the Phoenix'
'The Truth: Stage Adaptation'
'Love & Misadventure'

'Harry Potter and the Order of the Phoenix'

In [37]:
#Example usage:
import user_profiling
import os
import pandas as pd

def get_recommendation(user_id=None,book_title=None,top_n=10):

    if user_id and book_title:
        print('cannot have both user_id and book_title')
        exit(1)
    profiling=user_profiling.UserProfiling()

    top_n+=5


    if not os.path.exists('profiles.pkl'):
        profiling.create_profiles()

    df=pd.read_csv('data/book_names_final.csv')

    if book_title=='yes':
        book_title_input=input('Enter Title of the book:\n')
        book_id_series = df[df.book_title == book_title_input].book_id
        if not book_id_series.empty:
            book_id = book_id_series.values[0]  # Extract the book_id value
            print(f"Book ID: {book_id}")
        print(book_id)
    else:
        book_id=None
    
    recommendations, scores = profiling.recommend_books(user_id,book_id,top_n)
    if recommendations is None:
        return print('no recommendations found')
    titles=[]
    
    # i list(rec)
    for i in list(recommendations):
        title=df[df.book_id==i].book_title.values
        # print(title)
        try:
            title=title[0]
        except:
            continue
        
        
        
        titles.append(title)
    titles=titles[1:top_n-4]

    d=pd.DataFrame({'book_title':titles})
    print(d)
    # print(scores)

def main():
    get_recommendation(user_id=1,book_title=None)
    


In [38]:
main()

                                          book_title
0  Adapt and Plan for the New Abnormal of the COV...
1  Trump: Think Like a Billionaire: Everything Yo...
2  Brain Rules: 12 Principles for Surviving and T...
3                                      How We Decide
4                             Trump: How to Get Rich
5   The Checklist Manifesto: How to Get Things Right
6  The Fifth Discipline: The Art & Practice of Th...
7  Creativity, Inc.: Overcoming the Unseen Forces...
8  Made to Stick: Why Some Ideas Survive and Othe...
9  A Whole New Mind: Why Right-Brainers Will Rule...


In [6]:
import user_profiling
import os
import psycopg2
from dotenv import load_dotenv

def get_recommendation(user_id=None, book_title=None, top_n=10):
    if user_id and book_title:
        print('Cannot have both user_id and book_title')
        exit(1)

    profiling = user_profiling.UserProfiling()
    top_n += 5

    load_dotenv()

    connection_string = os.getenv('DATABASE_URL')

    # Connect to the Postgres database
    conn = psycopg2.connect(connection_string)

    # Create a cursor object
    cur = conn.cursor()
    schema_name = 'books_recommendation'
    table_name_name = 'books'

    if not os.path.exists('profiles.pkl'):
        profiling.create_profiles()

    if book_title == 'yes':
        book_title_input = input('Enter Title of the book:\n')
        # Execute SQL query to retrieve book_id based on book_title
        cur.execute(f"SELECT book_id FROM {schema_name}.{table_name_name} WHERE book_title = %s", (book_title_input,))
        book_id_row = cur.fetchone()
        if book_id_row:
            book_id = book_id_row[0]
            print(f"Book ID: {book_id}")
        else:
            print("Book not found")
            exit(1)
    else:
        book_id = None

    recommendations, scores = profiling.recommend_books(user_id, book_id, top_n)
    if recommendations is None:
        return print('No recommendations found')

    titles = []

    for i in list(recommendations):
        cur.execute(f"SELECT book_title FROM {schema_name}.{table_name_name} WHERE book_id = %s", (i,))
        title_row = cur.fetchone()
        if title_row:
            title = title_row[0]
            titles.append(title)

    titles = titles[1:top_n-5]

    for title in titles:
        print(title)

    cur.close()
    conn.close()

def main():
    get_recommendation(user_id=1, book_title=None)

if __name__ == "__main__":
    main()


Adapt and Plan for the New Abnormal of the COVID-19 Coronavirus Pandemic: Adapt and Plan for the New Abnormal of the COVID-19 Coronavirus Pandemic
Trump: Think Like a Billionaire: Everything You Need to Know About Success, Real Estate, and Life
Brain Rules: 12 Principles for Surviving and Thriving at Work, Home, and School
How We Decide
Trump: How to Get Rich
The Checklist Manifesto: How to Get Things Right
The Fifth Discipline: The Art & Practice of The Learning Organization
Creativity, Inc.: Overcoming the Unseen Forces That Stand in the Way of True Inspiration
Made to Stick: Why Some Ideas Survive and Others Die
